<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/yykim/yykim_TextRank_for_Sentence_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TextRank: 문서요약

In [71]:
import numpy as np


In [72]:
s1 = "딸기 바나나 사과 파인애플 수박"
s2 = "바나나 사과 딸기 포도"
s3 = "복숭아 수박"
s4 = "파인애플 사과 딸기 바나나"

In [73]:
# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()
token_s3 = s3.split()
token_s4 = s4.split()
doc = [token_s1,token_s2,token_s3,token_s4]

In [74]:
#문장간 유사도
similarity = {}
for i, v in enumerate(doc):
    for j in range(len(doc)):
        union = set(v).union(set(doc[j]))
        intersection = set(v).intersection(set(doc[j]))
        similarity.setdefault((i,j),len(intersection)/len(union))
similarity

{(0, 0): 1.0,
 (0, 1): 0.5,
 (0, 2): 0.16666666666666666,
 (0, 3): 0.8,
 (1, 0): 0.5,
 (1, 1): 1.0,
 (1, 2): 0.0,
 (1, 3): 0.6,
 (2, 0): 0.16666666666666666,
 (2, 1): 0.0,
 (2, 2): 1.0,
 (2, 3): 0.0,
 (3, 0): 0.8,
 (3, 1): 0.6,
 (3, 2): 0.0,
 (3, 3): 1.0}

In [75]:
graph = np.zeros((4,4))

In [76]:
for i in similarity:
    graph[i] = similarity[i]
for i in range(len(graph)):
    graph[(i,i)] = 0

In [77]:
graph

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]])

In [78]:
edge_weight =  np.zeros((4,4))
for i, v in enumerate(graph):
    edge_weight[i] = graph[i]/np.sum(v)

In [79]:
edge_weight #엣지가중치

array([[0.        , 0.34090909, 0.11363636, 0.54545455],
       [0.45454545, 0.        , 0.        , 0.54545455],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142857, 0.42857143, 0.        , 0.        ]])

In [80]:
def make_scores(graph):
    scores = np.zeros((4,))
    for i,v in enumerate(graph):
        for j, w in enumerate(graph):
            scores[i] += w[i]
    scores = (1-0.85) + 0.85*scores
    return scores

graph_score = make_scores(graph)
graph_score

array([1.39666667, 1.085     , 0.29166667, 1.34      ])

In [81]:
scores=make_scores(graph)

for iter in range(10):    # 10번 반복
    for i, v in enumerate(graph):
        for j,w in enumerate(graph):
            graph[(i,j)] = edge_weight[(i,j)]*scores[i]
    scores = make_scores(graph)              
    print(graph)
    print(scores)
    print("="*50)



[[0.         0.47613636 0.15871212 0.76181818]
 [0.49318182 0.         0.         0.59181818]
 [0.29166667 0.         0.         0.        ]
 [0.76571429 0.57428571 0.         0.        ]]
[1.46797835 1.04285877 0.2849053  1.30059091]
[[0.         0.50044717 0.16681572 0.80071547]
 [0.47402671 0.         0.         0.56883205]
 [0.2849053  0.         0.         0.        ]
 [0.74319481 0.5573961  0.         0.        ]]
[1.4268078  1.04916678 0.29179336 1.31411539]
[[0.         0.48641175 0.16213725 0.7782588 ]
 [0.47689399 0.         0.         0.57227279]
 [0.29179336 0.         0.         0.        ]
 [0.75092308 0.56319231 0.         0.        ]]
[1.44166887 1.04216345 0.28781666 1.29795185]
[[0.         0.49147802 0.16382601 0.78636484]
 [0.47371066 0.         0.         0.56845279]
 [0.28781666 0.         0.         0.        ]
 [0.74168677 0.55626508 0.         0.        ]]
[1.42773198 1.04058164 0.28925211 1.30159499]
[[0.         0.48672681 0.16224227 0.7787629 ]
 [0.47299165 

## Class로 만들기

In [123]:
class TextRank:
    def __init__(self, doc, max_iter=10, keysentence_number=1, DampingFactor=0.85, threshold=0.001 ): #s는 문서내 각각 토큰화된 문장을 리스트에 넣은 것
        self.max_iter = max_iter
        self.keysentence_number = keysentence_number
        self.link_graph = np.zeros((1,1))
        self.edge_graph = np.zeros((1,1))
        self.scores = np.ones((1,1))
        self.edge_count = []
        self.doc = doc
        self.DampingFactor = DampingFactor
        self.similarity ={}
        self.graph = np.zeros((1,1))
        self.threshold = threshold


    def _count_similarity(self):
        for i, v in enumerate(self.doc):
            for j in range(len(self.doc)):
                union = set(v).union(set(self.doc[j]))
                intersection = set(v).intersection(set(self.doc[j]))
                self.similarity.setdefault((i,j),len(intersection)/len(union))
        return self.similarity                
    
    def _make_graph(self):
        self.graph = np.zeros((len(self.doc),len(self.doc)))
        for i in self.similarity:
            self.graph[i] = self.similarity[i]
        for i in range(len(self.graph)):
            self.graph[(i,i)] = 0
        return self.graph

    def _make_edge_weight(self):
        self.edge_weight = np.zeros((len(self.doc),len(self.doc)))
        for i, v in enumerate(self.graph):
            self.edge_weight[i] = self.graph[i]/np.sum(v)
        return self.edge_weight

    def _make_scores(self):
        self.scores = np.zeros((len(self.doc),))
        for i,v in enumerate(self.graph):
            for j, w in enumerate(self.graph):
                self.scores[i] += w[i]
        self.scores = (1-self.DampingFactor) + self.DampingFactor*self.scores
        return self.scores
    
    def _learning(self):
        self.scores = self._make_scores()
        for iter in range(self.max_iter):
            before_scores = self.scores.copy()

            for i, v in enumerate(self.graph):
                for j,w in enumerate(self.graph):
                    self.graph[(i,j)] = self.edge_weight[(i,j)]*self.scores[i]
            
            self.scores = self._make_scores()

            if np.sum(np.square(self.scores-before_scores)) < self.threshold :
                break
        
    def text_rank(self): #스코어순서대로 정렬 및 해당 단어 보여주기
        self._count_similarity()
        self._make_edge_weight()
        self._make_scores()
        self._learning()

        sort = np.argsort(self.scores)[::-1]
        top = sort[0:self.keysentence_number]

        keysentence_ls = []
        for i in top :
            keysentence_ls.append(self.doc[i])
        return keysentence_ls

In [124]:
textrank = TextRank(doc)

In [125]:
textrank.text_rank()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


[['딸기', '바나나', '사과', '파인애플', '수박']]